# Web scraping a dataframe from a wikitable (Q1)

The first part of this assignment relates to scraping a wikipedia page for neighborhoods in Toronto. 

Step 1: Import necessary packages

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

Step 2: Request the webpage contents using the 'request' package

In [2]:
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(URL)

Step 3: Use 'BeautifulSoup' to parse the html into a BeautifulSoup object

In [3]:
soup = BeautifulSoup(page.content, 'html.parser')

Step 5: Locate on the page the information of interest by inspecting the page in a web browser. In this case we want the information contained in a wikitable. We can use our BeautifulSoup object to isolate that table.

In [4]:
myTable = soup.find('table',{'class':'wikitable sortable'})

Step 6: The table can be read into a pandas dataframe with the 'read_html' command by casting 'myTable' to a string. The first index [0] contains the dataframe. The procedure can be performed in one line of code:

In [5]:
df = pd.read_html(str(myTable))[0]
print(type(df))
df.head()

<class 'pandas.core.frame.DataFrame'>


,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Step 7: Per the instructions, we do not want to analyze rows with 'Not assigned' Boroughs. We can filter those rows out and reset the index with the .reset_index method.

In [6]:
df_filt = df[df['Borough']!='Not assigned'].reset_index(drop=True)
df_filt.head(10)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Note: the table is sorted according to 'Postal Code'. The appearance is different than the table in the assignment description but all the data is there. You can spot check values with: 

In [7]:
#df_filt[df_filt['Postal Code']=='M5G']

Step 8: We need to check if there are any neighborhoods with the designation 'Not assigned', if there are we should set them equal to the Borough.

In [8]:
#df_filt['Neighborhood'].unique()
#check_neighborhood = df_filt[df_filt['Neighborhood']=='Not assigned']
#check_neighborhood

df_filt['Neighborhood'].replace('Not assigned', df_filt['Borough'])

0                                              Parkwoods
1                                       Victoria Village
2                              Regent Park, Harbourfront
3                       Lawrence Manor, Lawrence Heights
4            Queen's Park, Ontario Provincial Government
                             ...                        
98         The Kingsway, Montgomery Road, Old Mill North
99                                  Church and Wellesley
100    Business reply mail Processing Centre, South C...
101    Old Mill South, King's Mill Park, Sunnylea, Hu...
102    Mimico NW, The Queensway West, South of Bloor,...
Name: Neighborhood, Length: 103, dtype: object

Per the assignment instructions I am replacing 'Not assigned' Neighborhoods with the Borough, but in this case you can run the commented out code and use the .unique() method or run the check_neighborhood lines and verify that there are no 'Not assigned' neighborhoods in this case. We can continue on.

Step 9: Print the number of rows using .shape:

In [9]:
print('The number of rows in my dataframe are {}'.format(df_filt.shape[0]))

The number of rows in my dataframe are 103


Summary: we have scraped a wikipage for the contents of a wikitable and read it into a dataframe. The dataframe was filtered and is closer to our final dataframe.



# Merging postal codes with lat/lon (Q2)
Now that we have the postal codes, we want to find the lat/lon values associated with the postal codes.

The geocoder script was taking a long time so I have uploaded the .csv from the submission page.

In [10]:
postal_codes = pd.read_csv('Geospatial_Coordinates.csv', sep=',')
postal_codes.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


We want to merge our filtered dataframe from above with the postal codes including lat/lon values. We can use the .merge method to do this. We want to 'left' merge, meaning we are adding the lat/lon columns to every sample in our original dataframe. This makes sure we don't remove any rows of data.

In [11]:
new_df = pd.merge(df_filt, postal_codes, how='left', on='Postal Code')
new_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Summary: we uploaded a table with the postal codes and lat/lon values into a dataframe. Then we merged on the postal codes to arrive at our final dataframe ready for analysis.

# Clustering Toronto Neighborhoods (Q3)

We start by importing some packages we will be using:

In [12]:
import folium
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 

For readability and ease, I have created two functions. The get_coord function takes in an address and user_agent and returns the lat/lon values. The add_markers function takes in our processed dataframe of neighborhoods and an empty folium map object and adds the neighborhoods to the map.

In [13]:
def get_coord(address, usr_agent):
    geolocator = Nominatim(user_agent=usr_agent)
    location = geolocator.geocode(address)
    print('The geographical coordinates of {} are {}, {}'.format(address, location.latitude, location.longitude))
    return location.latitude, location.longitude

def add_markers(df, folium_map):
    for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
        label = '{}, {}'.format(neighborhood, borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker([lat, lng], radius=5, popup=label, color='blue', fill=True, fill_color='#3186cc', fill_opacity=0.7, parse_html=False).add_to(folium_map)
    return folium_map

Next, I create the map of Toronto with neighborhoods:

In [14]:
#Generate Toronto map
latitude, longitude = get_coord('Toronto', 'to_explorer')               #Get lat/lon
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10) #Create map object
map_toronto = add_markers(new_df, map_toronto)                          #Add markers to map
map_toronto

The geographical coordinates of Toronto are 43.6534817, -79.3839347


I load in the getNearbyVenues function from the lab assignment, but I have added credentials as one of the inputs. I explain more on that below.

In [15]:
def getNearbyVenues(credentials, LIMIT, names, latitudes, longitudes, radius=500):
    CLIENT_ID, CLIENT_SECRET, VERSION = credentials.split(',')
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


Here I decided to keep a 'foursquare_credentials.txt' file in my working directory with the CLIENT_ID, CLIENT_SECRET, and VERSION. This way I can freely share and version control my scripts without including sensitive information. The text file should simply contain YOUR_CLIENT_ID,YOUR_CLIENT_SECRET,VERSION separated by commas. If you don't want to make this file you can comment this out and uncomment the other code and replace the variables with your information.

In [16]:
with open('foursquare_credentials.txt') as cred:
        credentials = cred.read()
        
#Alternative credential code (VERSION = 20180605)
#credentials = 'YOUR_CLIENT_ID,YOUR_CLIENT_SECRET,VERSION'

Here I call the getNearbyVenues function and output some basic information about the results, same as we did in the lab.

In [17]:
LIMIT = 100
toronto_venues = getNearbyVenues(credentials, LIMIT,
                               names=new_df['Neighborhood'],
                               latitudes=new_df['Latitude'],
                               longitudes=new_df['Longitude']
                              )

print('Toronto venues: {}'.format(toronto_venues.shape))
print(toronto_venues.groupby('Neighborhood').count())
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

Next, I load up the functions we used in the lab to find the top venues and return the most common venues.

In [18]:
def find_top_venues(city_grouped, num_top_venues):
    for hood in city_grouped['Neighborhood']:
        print("----"+hood+"----")
        temp = city_grouped[city_grouped['Neighborhood'] == hood].T.reset_index()
        temp.columns = ['venue','freq']
        temp = temp.iloc[1:]
        temp['freq'] = temp['freq'].astype(float)
        temp = temp.round({'freq': 2})
        print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
        print('\n')

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now with the venues in hand we can convert our categorical variables into dummy binary variables with the pd.get_dummies function.

In [20]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

Now we want to add back in the neighborhoods to our dummy dataframe and analyze the dataframe for top venues. We will use these results to analyze the clusters later.

In [21]:
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

#Group by neighborhood
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

find_top_venues(toronto_grouped, 5)

#Find top 10 venues for each neighborhood
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)


----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1             Breakfast Spot  0.25
2                     Lounge  0.25
3               Skating Rink  0.25
4              Metro Station  0.00


----Alderwood, Long Branch----
          venue  freq
0   Pizza Place  0.22
1  Skating Rink  0.11
2   Coffee Shop  0.11
3           Pub  0.11
4  Dance Studio  0.11


----Bathurst Manor, Wilson Heights, Downsview North----
            venue  freq
0     Coffee Shop  0.11
1            Bank  0.11
2     Bridal Shop  0.05
3  Ice Cream Shop  0.05
4           Diner  0.05


----Bayview Village----
                 venue  freq
0                 Bank  0.25
1   Chinese Restaurant  0.25
2                 Café  0.25
3  Japanese Restaurant  0.25
4          Yoga Studio  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0          Restaurant  0.08
1      Sandwich Place  0.08
2  Italian Restaurant  0.08
3         Coffee Shop  0.08
4         Pizza Place

4        Massage Studio   0.0


----Milliken, Agincourt North, Steeles East, L'Amoreaux East----
                             venue  freq
0                       Playground  0.33
1                             Park  0.33
2                      Coffee Shop  0.33
3               Mexican Restaurant  0.00
4  Molecular Gastronomy Restaurant  0.00


----Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West----
                    venue  freq
0          Discount Store  0.06
1  Thrift / Vintage Store  0.06
2             Flower Shop  0.06
3    Fast Food Restaurant  0.06
4          Sandwich Place  0.06


----Moore Park, Summerhill East----
                             venue  freq
0                            Trail  0.33
1                     Tennis Court  0.33
2                           Lawyer  0.33
3                      Yoga Studio  0.00
4  Molecular Gastronomy Restaurant  0.00


----New Toronto, Mimico South, Humber Bay Shores----
                venue

Now that we have the top venues for our neighborhoods, we can perform our cluster analysis. Below we perform the cluster analysis for 5 clusters and merge the results with the top neighborhood venues that we determined above. The merged dataframe is printed for review.

In [22]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# create a toronto_merged dataframe from our neighborhoods + lat/lon dataframe (new_df)
toronto_merged = new_df

# merge toronto_grouped with toronto_merged to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

print(toronto_merged.head()) # Cluster analysis results merged with top venues

  Postal Code           Borough                                 Neighborhood  \
0         M3A        North York                                    Parkwoods   
1         M4A        North York                             Victoria Village   
2         M5A  Downtown Toronto                    Regent Park, Harbourfront   
3         M6A        North York             Lawrence Manor, Lawrence Heights   
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   

    Latitude  Longitude  Cluster Labels 1st Most Common Venue  \
0  43.753259 -79.329656             1.0     Food & Drink Shop   
1  43.725882 -79.315572             0.0          Hockey Arena   
2  43.654260 -79.360636             0.0           Coffee Shop   
3  43.718518 -79.464763             0.0        Clothing Store   
4  43.662301 -79.389494             0.0           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue  4th Most Common Venue  \
0                  Park             Drugstore           

Here we will create the map and add markers with colors indicating each cluster.

In [23]:
# create map
map_toronto_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    if np.isnan(cluster):
        cluster = 0
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=0.7).add_to(map_toronto_clusters)
       


In [24]:
map_toronto_clusters

You can explore the different clusters with the following line of code, changing the float from 0-4.

In [25]:
toronto_merged[toronto_merged['Cluster Labels']==0.0]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Hockey Arena,French Restaurant,Coffee Shop,Portuguese Restaurant,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0.0,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Ice Cream Shop,French Restaurant,Gym / Fitness Center
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0.0,Clothing Store,Accessories Store,Women's Store,Coffee Shop,Event Space,Miscellaneous Shop,Carpet Store,Furniture / Home Store,Athletics & Sports,Boutique
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0.0,Coffee Shop,Diner,Yoga Studio,Hobby Shop,Burrito Place,Sandwich Place,Smoothie Shop,Café,Italian Restaurant,Beer Bar
7,M3B,North York,Don Mills,43.745906,-79.352188,0.0,Gym,Café,Coffee Shop,Japanese Restaurant,Beer Store,Restaurant,Sandwich Place,Athletics & Sports,Supermarket,Caribbean Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,0.0,Coffee Shop,Pub,Pizza Place,Convenience Store,Café,Restaurant,Bakery,Italian Restaurant,Flower Shop,Deli / Bodega
97,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280,0.0,Coffee Shop,Café,Hotel,Japanese Restaurant,Restaurant,Gym,Steakhouse,Asian Restaurant,Seafood Restaurant,Deli / Bodega
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0.0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Yoga Studio,Hotel,Men's Store,Pizza Place,Pub
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,0.0,Light Rail Station,Auto Workshop,Comic Shop,Park,Pizza Place,Restaurant,Burrito Place,Brewery,Skate Park,Smoke Shop


By analyzing the clusters I've simplified the results below:

Cluster 0 - Restaurants and coffee shops. The vast majority of the city falls in this category. The venues reflect typical metro areas 'city living'.

Cluster 1 - Parks, playgrounds, discount stores and dance studios. Possibly lower income neighborhoods.

Cluster 2 - Golf courses and department stores. Industrial centers with golf courses.

Cluster 3 - Baseball fields and department stores. Industrial centers with baseball fields.

Cluster 4 - Parks, playgrounds, restaurants, and high end stores. Possibly higher income neighborhoods.

Summary: we have scraped a wikipedia page for Toronto neighborhoods and used geopy to attach lat/lon values to each neighborhood. We then made API calls to foursquare.com to locate venues in each neighborhood. From the venue categories we have performed a cluster analysis on the neighborhoods to group them by similar venues. We have then created a map of Toronto with clustered neighborhoods.